# Lakebase Installation & Setup

This notebook demonstrates how to create and manage a Databricks Lakebase database instance using the Databricks SDK.

In [ ]:
# Install required packages
# !pip install databricks-sdk psycopg2-binary

In [1]:
import time
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
from databricks.sdk.errors import NotFound

## Configuration

Update these variables to match your setup:

In [12]:
# Configuration
INSTANCE_NAME = "rb-demo-lakebase"
PROFILE = "az-demo"
CAPACITY = "CU_1"

In [14]:
# Initialize Workspace client
w = WorkspaceClient(profile="az-demo")
print(f"✅ Connected to workspace: {w.config.host}")

ValueError: default auth: metadata-service: Metadata Service returned empty token. Config: host=https://e2-demo-field-eng.cloud.databricks.com, token=***, profile=az-demo, auth_type=metadata-service, serverless_compute_id=auto, metadata_service_url=***. Env: DATABRICKS_HOST, DATABRICKS_AUTH_TYPE, DATABRICKS_SERVERLESS_COMPUTE_ID, DATABRICKS_METADATA_SERVICE_URL

## Check if Instance Exists

In [7]:
try:
    existing_instance = w.database.get_database_instance(INSTANCE_NAME)
    print(f"✅ Instance '{INSTANCE_NAME}' already exists!")
    print(f"   State: {existing_instance.state}")
    print(f"   DNS: {existing_instance.read_write_dns}")
    
except NotFound:
    print(f"📝 Instance '{INSTANCE_NAME}' does not exist. Creating new instance...")

📝 Instance 'rb-demo-lakebase' does not exist. Creating new instance...


## Create New Instance (if needed)

In [ ]:
try:
    # Create database instance
    instance = w.database.create_database_instance(
        DatabaseInstance(
            name=INSTANCE_NAME,
            capacity=CAPACITY
        )
    )
    
    print(f"🚀 Instance creation initiated: {instance.name}")
    print(f"   Capacity: {instance.capacity}")
    print("   ⏳ Provisioning in progress...")
    
except Exception as e:
    print(f"❌ Error creating instance: {e}")

## Wait for Instance to be Ready

In [ ]:
def wait_for_ready(instance_name, timeout_minutes=15):
    """Wait for instance to be fully provisioned"""
    print(f"⏳ Waiting for '{instance_name}' to be ready...")
    
    start_time = time.time()
    timeout_seconds = timeout_minutes * 60
    
    while time.time() - start_time < timeout_seconds:
        try:
            instance = w.database.get_database_instance(instance_name)
            
            if instance.state and 'AVAILABLE' in str(instance.state):
                elapsed = (time.time() - start_time) / 60
                print(f"✅ Instance ready! (took {elapsed:.1f} minutes)")
                return instance
            
            elapsed = int(time.time() - start_time)
            print(f"   [{elapsed}s] State: {instance.state}")
            time.sleep(30)
            
        except Exception as e:
            print(f"   ⚠️  Error: {e}")
            time.sleep(30)
    
    raise TimeoutError(f"Instance not ready after {timeout_minutes} minutes")

In [ ]:
# Wait for instance to be ready (only if we just created it)
try:
    ready_instance = wait_for_ready(INSTANCE_NAME)
    
    print("\n📋 Instance Details:")
    print(f"   Name: {ready_instance.name}")
    print(f"   State: {ready_instance.state}")
    print(f"   DNS: {ready_instance.read_write_dns}")
    print(f"   PostgreSQL: {ready_instance.pg_version}")
    
except TimeoutError as e:
    print(f"❌ {e}")
except Exception as e:
    print(f"ℹ️  Instance may still be provisioning: {e}")

## Next Steps

Once your instance is ready, proceed to the next notebook to set up the database schema and sample data.